# Re-ranking

Amazon Bedrock은 쿼리 시 검색된 결과의 관련성을 향상시키기 위해 사용할 수 있는 재순위 모델에 대한 액세스를 제공합니다. 
재순위 모델은 청크와 쿼리의 관련성을 계산하고 계산된 점수를 기반으로 결과를 재정렬합니다. 
재순위 모델을 사용하면 쿼리에 답변하는 데 더 적합한 응답을 반환할 수 있습니다.

재순위 모델은 쿼리를 기반으로 관련성 신호를 식별한 다음 해당 신호를 사용하여 문서의 순위를 매기도록 훈련됩니다. 
이로 인해 모델은 더 관련성 있고 정확한 결과를 제공할 수 있습니다.

RAG(Retrieval Augmented Generation) 애플리케이션 구축을 위해 Amazon Bedrock Knowledge Bases를 사용하는 경우, Retrieve 또는 RetrieveAndGenerate 작업을 호출할 때 재순위 모델을 사용하세요. 
재순위 결과는 Amazon Bedrock Knowledge Bases가 결정하는 기본 순위를 재정의합니다.

이 노트북은 Amazon Bedrock Knowledge Bases와 함께 재순위 모델을 사용하는 방법을 보여주며, 
Rerank API를 통해 RAG 애플리케이션의 정확성과 관련성을 더욱 향상시키는 데 도움이 됩니다. 
재순위 모델을 사용하면 더 적지만 더 관련성 있는 결과를 검색할 수 있습니다. 
이러한 결과를 응답 생성에 사용하는 기반 모델에 제공함으로써 비용과 지연 시간을 줄일 수도 있습니다.

Amazon Bedrock Knowledge Bases와 함께 재순위 모델을 구현하고 활용하는 방법을 예제 사용 사례를 통해 살펴보겠습니다.

## 1. Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되었는지 확인하고 Bedrock에 연결해야 합니다.
pip 종속성 오류가 있다면 무시하세요 (라이브러리 설치 중 오류가 보이더라도)

In [71]:
%pip install --upgrade pip --quiet
%pip install -r ../requirements.txt --no-deps --quiet
%pip install -r ../requirements.txt --upgrade --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.3.1 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.39.16 which is incompatible.
autogluon-common 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 which is incompatible.
autogluon-core 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 which is incompatible.
autogluon-features 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 which is incompatible.
autogluon-multimodal 1.3.1 requires nltk<3.9,>=3.4.5, but you have nltk 3.9.1 which is incompatible.
autogluon-multimodal 1.3.1 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.1 whi

In [72]:
%pip install ragas==0.1.9 --quiet

Note: you may need to restart the kernel to use updated packages.


In [73]:
%pip install --upgrade boto3

Note: you may need to restart the kernel to use updated packages.


In [74]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [75]:
import boto3
print(boto3.__version__)

1.39.14


In [76]:
import warnings
warnings.filterwarnings('ignore')

이 코드는 설정의 일부이며 다음 용도로 사용됩니다:
- 상위 디렉토리를 파이썬 시스템 경로에 추가
- 나중에 실행하는 데 필요한 utils에서 사용자 정의 모듈(BedrockStructuredKnowledgeBase)을 가져옴

In [77]:
import os
import sys
import time
import boto3
import logging
import pprint
import json

# Set the path to import module
from pathlib import Path
current_path = Path().resolve()
current_path = current_path.parent
if str(current_path) not in sys.path:
    sys.path.append(str(current_path))
# Print sys.path to verify
# print(sys.path)

from utils.knowledge_base import BedrockKnowledgeBase

In [78]:
#Clients
s3_client = boto3.client('s3')
sts_client = boto3.client('sts')
session = boto3.session.Session(region_name = 'us-west-2')
region =  session.region_name
account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime') 
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)
region, account_id

('us-west-2', '211125368524')

In [79]:
import time

# Get the current timestamp
current_time = time.time()

# Format the timestamp as a string
timestamp_str = time.strftime("%Y%m%d%H%M%S", time.localtime(current_time))[-7:]
# Create the suffix using the timestamp
suffix = f"{timestamp_str}"
knowledge_base_name = 'reranking-kb'
knowledge_base_description = "Knowledge Base for re-ranking."
bucket_name = f'{knowledge_base_name}-{suffix}'
foundation_model = "anthropic.claude-3-sonnet-20240229-v1:0"

# Define data sources
data_source=[{"type": "S3", "bucket_name": bucket_name}]

## 2 - Create knowledge bases with fixed chunking strategy
csv 형식의 비디오 게임 데이터를 저장하기 위한 [Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/)를 생성하는 것부터 시작하겠습니다. 
Knowledge Bases를 사용하면 [Amazon OpenSearch Serverless](https://aws.amazon.com/opensearch-service/features/serverless/), [Amazon Aurora](https://aws.amazon.com/rds/aurora/), [Pinecone](http://app.pinecone.io/bedrock-integration), [Redis Enterprise]() 및 [MongoDB Atlas]()를 포함한 다양한 벡터 데이터베이스와 통합할 수 있습니다. 이 예제에서는 지식 베이스를 Amazon OpenSearch Serverless와 통합합니다. 
이를 위해 지식 베이스와 모든 전제 조건을 생성하는 도우미 클래스 `BedrockKnowledgeBase`를 사용합니다:

1. IAM 역할 및 정책
2. S3 버킷
3. Amazon OpenSearch Serverless 암호화, 네트워크 및 데이터 액세스 정책
4. Amazon OpenSearch Serverless 컬렉션
5. Amazon OpenSearch Serverless 벡터 인덱스
6. 지식 베이스
7. 지식 베이스 데이터 소스

고정 청킹 전략을 사용하여 지식 베이스를 생성합니다.

아래 매개변수 값을 변경하여 다른 청킹 전략을 선택할 수 있습니다:
```
"chunkingStrategy": "FIXED_SIZE | NONE | HIERARCHICAL | SEMANTIC"
```

In [80]:
knowledge_base_metadata = BedrockKnowledgeBase(
    kb_name=f'{knowledge_base_name}-{suffix}',
    kb_description=knowledge_base_description,
    data_sources=data_source, 
    chunking_strategy = "FIXED_SIZE", 
    suffix = suffix
)

Step 1 - Creating or retrieving S3 bucket(s) for Knowledge Base documents
['reranking-kb-9225030']
buckets_to_check:  ['reranking-kb-9225030']
Creating bucket reranking-kb-9225030
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_9225030) and Policies
Step 3a - Creating OSS encryption, network and data access policies
Step 3b - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '318',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Tue, 29 Jul 2025 22:50:33 '
                                                 'GMT',
                                         'x-amzn-requestid': '10624b3b-65fd-4c44-a4d0-1b5cb8228e60'},
                        'HTTPStatusCode': 200,
                        'RequestId': '10624b3b-65

[2025-07-29 22:52:04,406] p5443 {base.py:258} INFO - PUT https://jhb81qgbkyfduf80qile.us-west-2.aoss.amazonaws.com:443/bedrock-sample-rag-index-9225030 [status:200 request:0.436s]



Creating index:
{ 'acknowledged': True,
  'index': 'bedrock-sample-rag-index-9225030',
  'shards_acknowledged': True}
Step 4 - Will create Lambda Function if chunking strategy selected as CUSTOM
Not creating lambda function as chunking strategy is FIXED_SIZE
Step 5 - Creating Knowledge Base
{ 'createdAt': datetime.datetime(2025, 7, 29, 22, 53, 4, 533963, tzinfo=tzlocal()),
  'description': 'Knowledge Base for re-ranking.',
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:211125368524:knowledge-base/QENX4YMJTL',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'QENX4YMJTL',
  'name': 'reranking-kb-9225030',
  'roleArn': 'arn:aws:iam::211125368524:role/AmazonBedrockExecutionRoleForKnowledgeBase_9225030',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { '

## 2.1 데이터셋을 Amazon S3에 업로드
이제 지식 베이스를 생성했으니, `Octank financial 10K` 보고서 데이터셋으로 채워보겠습니다. Knowledge Base 데이터 소스는 연결된 S3 버킷에서 데이터를 사용할 수 있어야 하며, `StartIngestionJob` 호출을 사용하여 데이터의 변경 사항을 지식 베이스와 동기화할 수 있습니다. 이 예제에서는 우리의 헬퍼 클래스를 통해 API의 [boto3 추상화](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent/client/start_ingestion_job.html)를 사용할 것입니다.

먼저 dataset 폴더에 있는 메뉴 데이터를 s3에 업로드해보겠습니다.

In [81]:
import os

def upload_directory(path, bucket_name):
    for root, dirs, files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root, file)
            if file not in ["LICENSE", "NOTICE", "README.md"]:
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload, bucket_name, file)
            else:
                print(f"Skipping file {file_to_upload}")

upload_directory("../synthetic_dataset", bucket_name)


uploading file ../synthetic_dataset/2024_population.pdf to reranking-kb-9225030
uploading file ../synthetic_dataset/.ipynb_checkpoints/2024_population-checkpoint.pdf to reranking-kb-9225030


이제 수집 작업을 시작합니다.

In [82]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base_metadata.start_ingestion_job()

job 1 started successfully

{ 'dataSourceId': 'DUC2HH4QJF',
  'ingestionJobId': 'UXKEV5YSKU',
  'knowledgeBaseId': 'QENX4YMJTL',
  'startedAt': datetime.datetime(2025, 7, 29, 22, 56, 17, 534583, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 2,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 2},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 29, 22, 56, 41, 245416, tzinfo=tzlocal())}
........................................

마지막으로 나중에 솔루션을 테스트하기 위해 Knowledge Base ID를 저장합니다.

In [83]:
kb_id = knowledge_base_metadata.get_knowledge_base_id()

'QENX4YMJTL'


## 3. Evaluate the relevance of query responses with and without Re-ranking (using Ragas)

생성, 평가 및 재순위를 위한 모델 정의

In [84]:
from langchain.llms.bedrock import Bedrock
from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings

bedrock_client = boto3.client('bedrock-runtime')

TEXT_GENERATION_MODEL_ID = "anthropic.claude-3-haiku-20240307-v1:0"
EVALUATION_MODEL_ID = "anthropic.claude-3-sonnet-20240229-v1:0"
EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"

# Reranker model: there are two reranker models available at launch
AMAZON_RERANKER_MODEL_ID = "amazon.rerank-v1:0"
COHERE_RERANKER_MODEL_ID = "cohere.rerank-v3-5:0"


llm_for_evaluation = ChatBedrock(model_id=EVALUATION_MODEL_ID, client=bedrock_client)
bedrock_embeddings = BedrockEmbeddings(model_id=EMBEDDING_MODEL_ID, client=bedrock_client)


#### 3.1 Update Knowledge Bases execution role

In [85]:
# Before using autogenerated filters - update the knowledge base execution IAM role with right permissions

iam = boto3.resource('iam')
client = boto3.client('iam')

def get_attached_policies(role_name):
    response = client.list_attached_role_policies(RoleName=role_name)
    attached_policies = response['AttachedPolicies']
    return attached_policies

# get the knowledge base IAM role name
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb_id)
role_arn = get_kb_response['knowledgeBase']['roleArn']
role_name = role_arn.split('/')[-1]

# get attached policies
attached_policies = get_attached_policies(role_name)
attached_policies

def update_kb_execution_role(attached_policies, region_name):
    
    for policy in attached_policies:

        print(policy['PolicyArn'])
        policy_name = policy['PolicyName']
        policy_arn = policy['PolicyArn']

        if 'FoundationModel' in policy_arn:
            print('Updating FoundationModel policy: ',policy_arn)
            policy = iam.Policy(policy_arn)
            version = policy.default_version
            policyJson = version.document
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}')
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{EVALUATION_MODEL_ID}')  
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{AMAZON_RERANKER_MODEL_ID}') 
            policyJson['Statement'][0]['Resource'].append(f'arn:aws:bedrock:{region}::foundation-model/{COHERE_RERANKER_MODEL_ID}') 
        
            client.detach_role_policy(RoleName=role_name,
                PolicyArn=policy_arn)
            
            response = client.delete_policy(
                PolicyArn=policy_arn
            )
            print(response)
           
            response = client.create_policy(
            PolicyName= policy_name,
            PolicyDocument=json.dumps(policyJson)
            )
            print(response)
        
        client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

In [86]:
update_kb_execution_role(attached_policies, region)
# time.sleep(30)

arn:aws:iam::211125368524:policy/AmazonBedrockCloudWatchPolicyForKnowledgeBase_9225030
arn:aws:iam::211125368524:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_9225030
Updating FoundationModel policy:  arn:aws:iam::211125368524:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_9225030
{'ResponseMetadata': {'RequestId': 'ffb621b5-d91c-451c-9fa1-636ec083eaf7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 29 Jul 2025 22:58:20 GMT', 'x-amzn-requestid': 'ffb621b5-d91c-451c-9fa1-636ec083eaf7', 'content-type': 'text/xml', 'content-length': '204'}, 'RetryAttempts': 0}}
{'Policy': {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_9225030', 'PolicyId': 'ANPATCKANO3GAKSYYYX42', 'Arn': 'arn:aws:iam::211125368524:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_9225030', 'Path': '/', 'DefaultVersionId': 'v1', 'AttachmentCount': 0, 'PermissionsBoundaryUsageCount': 0, 'IsAttachable': True, 'CreateDate': datetime.datetime(2025, 7, 29, 22, 58, 20, tz

#### 3.2 Customize retrieve and generate configuraion

In [87]:
def retrieve_and_generate(query, reranker_model=None, kb_id=None, TEXT_GENERATION_MODEL_ID=None, metadata_filters=None):
    
    # Prepare retrieval configuration
    retrieval_config = {
        "vectorSearchConfiguration": {
            "numberOfResults": 30 if reranker_model else 3
        }
    }

    if reranker_model:
        retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"] = {
            "type": "BEDROCK_RERANKING_MODEL",
            "bedrockRerankingConfiguration": {
                "modelConfiguration": {
                    "modelArn": f'arn:aws:bedrock:{region}::foundation-model/{reranker_model}',
                },
                "numberOfRerankedResults": 3
            }
        }

        if metadata_filters:
            retrieval_config["vectorSearchConfiguration"]["rerankingConfiguration"]["bedrockRerankingConfiguration"]["metadataConfiguration"] = {
                                                                "selectionMode" : "SELECTIVE",
                                                                "selectiveModeConfiguration" : {
                                                                    "fieldsToInclude": [{
                                                                        "fieldName": "year",
                                                                    }]
                                                                }
                                                            }
                    

    # Call the retrieve and generate API
    start = time.time()
    response = bedrock_agent_runtime_client.retrieve_and_generate(
        input={'text': query},
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': f'arn:aws:bedrock:{region}::foundation-model/{TEXT_GENERATION_MODEL_ID}',
                'retrievalConfiguration': retrieval_config,
            },
        }
    )
    time_spent = time.time() - start

    print(f"[Response] : {response['output']['text']}\n")
    print(f"[Invocation time] : {time_spent}\n")

    return response


#### 3.3 Prepare dataset for evaluation

In [136]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness
)

#specify the metrics here
metrics = [
    answer_correctness
]

questions = [
    "2024년 기준으로 지역별 고령화율, 1인가구 비율, 주택노후도, 빈집 비율 간의 상관관계를 분석해주세요."
]
ground_truths = [
    "각 지표간 상관관계와 지역별 특성을 분석하면 다음과 같습니다:\
    지표간 상관관계\
    고령화율과 1인가구 비율:\
    강한 양의 상관관계 확인\
    전남(고령화율 26.3%, 1인가구 37.7%)\
    경북(고령화율 24.9%, 1인가구 38.9%)\
    주택노후도와 빈집 비율:\
    매우 강한 양의 상관관계\
    노후주택(30년 이상) 비율 상위 지역:\
    전남 42.8%\
    경북 37.7%\
    대전 36.5%\
    지역유형별 특성\
    [도시형]\
    특징:\
    낮은 고령화율\
    높은 1인가구 비율\
    낮은 빈집 비율\
    해당지역: 서울, 세종 등\
    주요 지표:\
    서울: 1인가구 39.9%, 고령화율 19.2%\
    세종: 1인가구 32.9%, 고령화율 11.1%\
    [농촌형]\
    특징:\
    높은 고령화율\
    높은 빈집 비율\
    높은 주택노후도\
    해당지역: 전남, 경북 등\
    주요 지표:\
    전남: 고령화율 26.3%, 빈집비율 15.0%\
    경북: 고령화율 24.9%, 빈집비율 12.5%\
    정책적 시사점\
    도시지역:\
    청년 1인가구 주거지원 정책 필요\
    소형주택 공급 확대\
    주거비 부담 완화 정책\
    농촌지역:\
    빈집 정비사업 시급\
    고령자 맞춤형 주거환경 개선\
    귀농귀촌 지원을 통한 인구유입 필요\
    수도권:\
    주택공급 확대 필요\
    1인가구 맞춤형 주거정책\
    주거비 안정화 정책"
]

In [137]:
def prepare_eval_dataset(questions, ground_truths, kb_id=None, TEXT_GENERATION_MODEL_ID=None, reranker_model=None, metadata_filters = None):
    answers = []
    contexts = []
    
    for query in questions:
        response = retrieve_and_generate(
            query,
            reranker_model=reranker_model,
            kb_id=kb_id,
            TEXT_GENERATION_MODEL_ID=TEXT_GENERATION_MODEL_ID,
            metadata_filters=metadata_filters
        )
        
        answers.append(response["output"]["text"])
        
        context_group = []
        for citation in response["citations"]:
            context_group.extend([
                ref["content"]["text"]
                for ref in citation["retrievedReferences"]
                if "content" in ref and "text" in ref["content"]
            ])
        contexts.append(context_group)
        time.sleep(15)

    # Create dictionary
    data = {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
        "ground_truth": ground_truths
    }

    # Convert dict to dataset
    dataset = Dataset.from_dict(data)
    return dataset


#### 3.4 Evaluate dataset - without re-ranker

In [138]:
without_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=None)

[Response] : 2024년 기준 지역별 주택 노후도와 빈집 비율 간의 상관관계를 분석해보면 다음과 같습니다:

전남, 전북, 경북 등 노후주택 비율이 높은 지역에서 빈집 비율도 높게 나타나고 있습니다. 반면 경기, 제주 등 노후주택 비율이 상대적으로 낮은 지역에서는 빈집 비율도 낮은 편입니다. 이를 통해 주택 노후도와 빈집 비율 간에는 양의 상관관계가 있음을 알 수 있습니다. 그러나 지역별 고령화율, 1인가구 비율과 주택 노후도, 빈집 비율 간의 상관관계는 명확하게 드러나지 않습니다. 제주도의 경우 고령화율과 1인가구 비율이 상대적으로 낮지만 빈집 비율은 높은 편이며, 경북의 경우 고령화율과 1인가구 비율이 높은 편이지만 빈집 비율은 중간 수준입니다. 따라서 지역별 고령화율, 1인가구 비율과 주택 노후도, 빈집 비율 간의 상관관계는 명확하지 않다고 볼 수 있습니다.

[Invocation time] : 4.8748674392700195



In [139]:
without_reranker_result = evaluate(
    dataset=without_reranker_dataset,
    metrics=metrics,
    llm=llm_for_evaluation,
    embeddings=bedrock_embeddings,
)

without_reranker_result_df = without_reranker_result.to_pandas()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[2025-07-29 23:35:24,204] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-29 23:35:28,931] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-29 23:35:37,094] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response


#### 3.5 Evaluate dataset - with re-ranker

In [140]:
with_reranker_dataset = prepare_eval_dataset(questions, ground_truths, kb_id, TEXT_GENERATION_MODEL_ID, reranker_model=AMAZON_RERANKER_MODEL_ID)

[Response] : 2024년 기준 지역별 고령자 1인가구 비율을 살펴보면, 전남이 16.1%로 가장 높고, 경북 14.3%, 전북 14.1% 순으로 나타났습니다. 이는 전남, 경북, 전북 지역의 고령화율이 상대적으로 높기 때문인 것으로 보입니다. 그러나 이번 질문에서는 고령화율, 1인가구 비율, 주택노후도, 빈집 비율 간의 상관관계를 분석하라고 하였는데, 제공된 검색 결과에는 주택노후도와 빈집 비율에 대한 정보가 없어 이들 간의 상관관계를 분석하기는 어려웠습니다.

[Invocation time] : 4.52137017250061



In [141]:
with_reranker_result = evaluate(
dataset=with_reranker_dataset,
metrics=metrics,
llm=llm_for_evaluation,
embeddings=bedrock_embeddings,
)

with_reranker_result_df = with_reranker_result.to_pandas()

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

[2025-07-29 23:36:10,933] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-29 23:36:17,287] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response
[2025-07-29 23:36:30,416] p5443 {bedrock.py:940} INFO - Using Bedrock Invoke API to generate response


#### 3.5 Prepare Comparison data frame

In [142]:
import pandas as pd

# Create the side-by-side DataFrame
comparison_df = pd.DataFrame({
    'question': without_reranker_result_df['question'],
    'without_reranker_answer': without_reranker_result_df['answer'],
    'with_reranker_answer': with_reranker_result_df['answer'],
    
    'without_reranker_answer_correctness': without_reranker_result_df['answer_correctness'],
    'with_reranker_answer_correctness': with_reranker_result_df['answer_correctness'],
    })

In [143]:
pd.options.display.max_colwidth = 1000
comparison_df

,question,without_reranker_answer,with_reranker_answer,without_reranker_answer_correctness,with_reranker_answer_correctness
0,"2024년 기준으로 지역별 고령화율, 1인가구 비율, 주택노후도, 빈집 비율 간의 상관관계를 분석해주세요.","2024년 기준 지역별 주택 노후도와 빈집 비율 간의 상관관계를 분석해보면 다음과 같습니다:\n\n전남, 전북, 경북 등 노후주택 비율이 높은 지역에서 빈집 비율도 높게 나타나고 있습니다. 반면 경기, 제주 등 노후주택 비율이 상대적으로 낮은 지역에서는 빈집 비율도 낮은 편입니다. 이를 통해 주택 노후도와 빈집 비율 간에는 양의 상관관계가 있음을 알 수 있습니다. 그러나 지역별 고령화율, 1인가구 비율과 주택 노후도, 빈집 비율 간의 상관관계는 명확하게 드러나지 않습니다. 제주도의 경우 고령화율과 1인가구 비율이 상대적으로 낮지만 빈집 비율은 높은 편이며, 경북의 경우 고령화율과 1인가구 비율이 높은 편이지만 빈집 비율은 중간 수준입니다. 따라서 지역별 고령화율, 1인가구 비율과 주택 노후도, 빈집 비율 간의 상관관계는 명확하지 않다고 볼 수 있습니다.","2024년 기준 지역별 고령자 1인가구 비율을 살펴보면, 전남이 16.1%로 가장 높고, 경북 14.3%, 전북 14.1% 순으로 나타났습니다. 이는 전남, 경북, 전북 지역의 고령화율이 상대적으로 높기 때문인 것으로 보입니다. 그러나 이번 질문에서는 고령화율, 1인가구 비율, 주택노후도, 빈집 비율 간의 상관관계를 분석하라고 하였는데, 제공된 검색 결과에는 주택노후도와 빈집 비율에 대한 정보가 없어 이들 간의 상관관계를 분석하기는 어려웠습니다.",0.630729,0.722595


In [144]:
# Calculate average correctness
without_reranker_avg_correctness = without_reranker_result_df['answer_correctness'].mean()
with_reranker_avg_correctness = with_reranker_result_df['answer_correctness'].mean()

print(f"\nAverage Correctness without Reranker: {without_reranker_avg_correctness:.4f}")
print(f"Average Correctness with Reranker: {with_reranker_avg_correctness:.4f}")


Average Correctness without Reranker: 0.6307
Average Correctness with Reranker: 0.7226


### 2.7 Clean up
이 노트북에서 생성된 리소스를 삭제하려면 아래 셀의 주석을 해제하고 실행하세요.

In [145]:
print("===============================Knowledge base==============================")
#knowledge_base_metadata.delete_kb(delete_s3_bucket=True, delete_iam_roles_and_policies=True)

===============================Knowledge base==============================
